## Import

In [6]:
from skimage.segmentation import slic
import os
import radiomics
from radiomics import featureextractor
import numpy as np
import nrrd
import os
import numpy as np
import nrrd
import pandas as pd
from radiomics import featureextractor
from skimage.util import view_as_blocks
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import csv


import logging
# ... or set level for specific class
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)


## Funzioni Utili 2D

In [7]:

# Funzione per caricare il volume e la maschera
def carica_volume(file_path):
    return nrrd.read(file_path)[0]

def estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx):
    if not np.any(slice_maschera > 0):
        return None, None
    
    try:
        # Converti le slice da NumPy a SimpleITK
        sitk_slice_immagine = numpy_to_sitk(slice_immagine)
        sitk_slice_maschera = numpy_to_sitk(slice_maschera)
        
        # Estrai le features usando l'estrattore di SimpleITK
        results = estrattore.execute(sitk_slice_immagine, sitk_slice_maschera)
        
        # Trasforma i risultati in un vettore di features
        feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                                   for key, value in results.items() 
                                   if isinstance(value, (float, int, np.ndarray))])
        return feature_vector, results
    except Exception as e:
        print(f"Errore slice {idx}: {e}")  # Stampa l'errore specifico
        return None, None  # Non aggiungere la slice al CSV

# Funzione per creare il CSV con i nomi delle features
def crea_csv_features(pazienti_dati, features_name, csv_file):
    if not pazienti_dati:
        return

    headers = ['Paziente', 'Slice'] + features_name  # Nomi delle features

    with open(csv_file, mode='w', newline='') as file_csv:
        writer = csv.writer(file_csv)
        writer.writerow(headers)  # Scrivi l'intestazione

        for paziente, slice_idx, feature_vector in pazienti_dati:
            writer.writerow([paziente, slice_idx] + list(feature_vector))  # Scrivi i dati di ogni paziente


def numpy_to_sitk(array_numpy):
    array_numpy = np.expand_dims(array_numpy, axis = -1)
    return sitk.GetImageFromArray(array_numpy)


# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None


## Radiomica 2D

In [8]:
import csv
import numpy as np
import SimpleITK as sitk
import nrrd
from radiomics import featureextractor

# Percorsi delle directory
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Lista per i dati dei pazienti
pazienti_dati = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Ciclare attraverso le directory e abbinare i file NRRD
for paziente_folder in sorted(os.listdir(pazienti_dir)):
    paziente_path = os.path.join(pazienti_dir, paziente_folder)
    roi_path = os.path.join(roi_dir, paziente_folder)

    print(paziente_folder)

    if os.path.isdir(paziente_path) and os.path.isdir(roi_path):
        # Trova i file NRRD nelle directory del paziente e della maschera
        file_immagine_nrrd = trova_nrrd_file(paziente_path)
        file_maschera_nrrd = trova_nrrd_file(roi_path)
        
        if file_immagine_nrrd and file_maschera_nrrd:
            # Carica il volume e la maschera
            volume_immagine = carica_volume(file_immagine_nrrd)
            volume_maschera = carica_volume(file_maschera_nrrd)
            
            # Assicurati che volume_immagine e volume_maschera abbiano la stessa forma

            print(volume_immagine.shape)
            print(volume_maschera.shape)

            if volume_immagine.shape != volume_maschera.shape:
                print(f"Il volume dell'immagine e la maschera non hanno la stessa forma per il paziente {paziente_folder}")
                continue
            
            # Itera su ogni slice
            for idx in range(volume_immagine.shape[2]):
                slice_immagine = volume_immagine[:, :, idx]
                slice_maschera = volume_maschera[:, :, idx]

                # Estrai le features dalla slice
                feature_vector, results = estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx)
                if feature_vector is not None:
                    features_name = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]
                    pazienti_dati.append((paziente_folder, idx + 1, feature_vector))  # paziente_folder come identificativo paziente

# Ordina i dati prima di creare il CSV
pazienti_dati.sort(key=lambda x: (int(x[0]), x[1]))  # Ordina per paziente e poi per slice

# Crea il CSV con i dati estratti
crea_csv_features(pazienti_dati, features_name, 'C:\\Users\\bsbar\\Desktop\\Radiomica_2D.csv')

100
(512, 512, 529)
(512, 512, 529)
Errore slice 191: mask has too few dimensions (number of dimensions 1, minimum required 2)
101
(512, 512, 537)
(512, 512, 537)
102
(512, 512, 586)
(512, 512, 586)
103
(512, 512, 423)
(512, 512, 423)


c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\firstorder.py:297: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(numpy.absolute(percentileArray - numpy.nanmean(percentileArray, 1, keepdims=True)), 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm.py:258: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(ac, 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm.py:294: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(cp, 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm.py:312: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(cs, 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm.py:329: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(ct, 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm.py:344: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(cont, 1)
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\radiomics\glcm

104
(512, 512, 562)
(512, 512, 562)
Errore slice 251: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.
105
(512, 512, 507)
(512, 512, 507)
106
(512, 512, 600)
(512, 512, 600)
107
(512, 512, 536)
(512, 512, 536)
108
(512, 512, 488)
(512, 512, 488)
109
(512, 512, 542)
(512, 512, 542)
110
(512, 512, 535)
(512, 512, 535)
111
(512, 512, 608)
(512, 512, 608)
112
(512, 512, 615)
(512, 512, 615)
113
(512, 512, 509)
(512, 512, 509)
114
(512, 512, 513)
(512, 512, 513)
Errore slice 230: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.
115
(512, 512, 525)
(512, 512, 525)
Errore slice 181: mask has too few dimensions (number of dimensions 1, minimum required 2)
116
(512, 512, 502)
(512, 512, 502)
117
(512, 512, 582)
(512, 512, 582)
118
(512, 512, 476)
(512, 512, 476)
119
(512, 512, 532)
(512, 512, 532)
12
(512, 512, 512)
(512, 512, 512)
120
(512, 512, 555)
(512, 512, 555)
123
(512, 512, 567)
(512, 512, 567)
Errore slice 255: mask has

## Features Radiomiche 3D

In [4]:
# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
dimension_mismatch = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per verificare le dimensioni delle immagini
def verifica_dimensioni(immagine_file, maschera_file):
    immagine, _ = nrrd.read(immagine_file)
    maschera, _ = nrrd.read(maschera_file)
    return immagine.shape == maschera.shape

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    print(paziente)
    
    # Verifica se la cartella esiste anche per le ROI
    roi_paziente_dir = os.path.join(roi_dir, paziente)
    paziente_dir = os.path.join(pazienti_dir, paziente)

    if os.path.isdir(roi_paziente_dir):
        # Trova i file immagine e maschera nella rispettiva cartella
        immagine_file = trova_nrrd_file(paziente_dir)
        maschera_file = trova_nrrd_file(roi_paziente_dir)
     
        if immagine_file and maschera_file:
            # Verifica se le dimensioni delle immagini e delle ROI corrispondono
            if verifica_dimensioni(immagine_file, maschera_file):
                # Aggiungi il nome del paziente alla lista
                patient_names.append(paziente)

                # Estrai le features radiomiche
                results = estrattore.execute(immagine_file, maschera_file)

                # Trasforma i risultati in un vettore di features
                feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                           for key, value in results.items() 
                           if isinstance(value, (float, int, np.ndarray))])
                
                # Aggiungi il vettore delle features alla lista
                feature_vectors.append(feature_vector)
                
            else:
                # Aggiungi il nome del paziente e un messaggio di errore alla lista delle dimensioni non corrispondenti
                dimension_mismatch.append(paziente)

# Trasforma la lista di vettori in un array numpy
feature_matrix = np.array(feature_vectors)

# Ora hai:
# - 'feature_matrix': una matrice dove ogni riga è il vettore di features di un paziente
# - 'patient_names': una lista dei nomi dei pazienti corrispondente
# - 'dimension_mismatch': una lista dei nomi dei pazienti con dimensioni non corrispondenti

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
12
120
123
124
126
127
128
129
133
135
136
137
138
139
141
142
144
146
147
149
15
150
153
155
158
159
16
161
163
166
168
169
17
170
171
175
176
178
182
183
188
189
19
190
193
197
199
200
205
22
23
24
25
26
27
29
30
31
33
35
36
38
39
40
41
42
43
44
46
47
48
5
50
52
53
54
56
57
58
59
60
61
62
64
65
68
69
70
71
74
75
76
78
79
81
82
84
85
86
87
88
89
90
91
92
94
95
96
98


In [5]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'Radiomica_3D.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

File CSV salvato come Radiomica_3D.csv


In [36]:
import radiomics
from radiomics import featureextractor, getFeatureClasses

# Ottieni le classi di feature disponibili
feature_classes = getFeatureClasses()
print("Feature classes disponibili:")
for feature_class_name in feature_classes:
    print(feature_class_name)

Feature classes disponibili:
firstorder
glcm
gldm
glrlm
glszm
ngtdm
shape
shape2D


## Funzioni utili BLOCCHI

In [3]:
# Funzione per visualizzare una sezione specifica di un blocco 3D
def visualizza_blocco(blocco, sezione=None):
    if sezione is None:
        sezione = blocco.shape[2] // 2  # Visualizza la sezione centrale per default
    
    # Visualizza la sezione specifica del blocco
    plt.imshow(blocco[:, :, 11], cmap='gray')
    plt.title(f"Blocco - Sezione {sezione}")
    plt.axis('off')  # Nascondi gli assi
    plt.show()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per caricare il volume 3D delle immagini
def carica_volume_paziente(cartella_paziente):
    sezioni = []  # Lista per conservare le singole sezioni 2D
    for file in sorted(os.listdir(cartella_paziente)):
        if file.endswith('.nrrd'):
            # Leggi la sezione 2D
            immagine, _ = nrrd.read(os.path.join(cartella_paziente, file))
            sezioni.append(immagine)  # Aggiungi la sezione alla lista
    # Stack delle sezioni per formare un volume 3D (asse delle sezioni = asse z)
    volume_3d = np.stack(sezioni, axis=-1)
    return volume_3d

# Funzione per suddividere il volume in blocchi 3D
def suddividi_volume(volume_3d, dimensione_blocco=(16, 16, 16)):
    # Verifica se le dimensioni del volume sono divisibili esattamente per il blocco
    shape_volume = np.array(volume_3d.shape)
    if np.any(shape_volume % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni del volume non sono divisibili esattamente per il blocco 3D.")
    
    # Usa view_as_blocks per suddividere il volume in blocchi 3D
    blocchi = view_as_blocks(volume_3d, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in una lista di blocchi singoli
    blocchi_risagomati = blocchi.reshape(-1, *dimensione_blocco)
    return blocchi_risagomati

# Funzione per convertire un array NumPy in un oggetto SimpleITK
def numpy_to_sitk(volume_numpy):
    return sitk.GetImageFromArray(volume_numpy)

# Funzione per verificare se la maschera ha valori non nulli
def verifica_maschera_valida(maschera):
    return np.any(maschera > 0)


def riduci_slice(volume_3d, target_slices=512):
    slice_correnti = volume_3d.shape[-1]
    
    if slice_correnti < target_slices:
        slice_da_aggiungere = target_slices - slice_correnti
        slice_inizio = slice_da_aggiungere // 2
        slice_fine = slice_da_aggiungere - slice_inizio
        padding = ((0, 0), (0, 0), (slice_inizio, slice_fine))
        volume_pad = np.pad(volume_3d, pad_width=padding, mode='constant', constant_values=0)
        return volume_pad

    elif slice_correnti > target_slices:
        slice_da_rimuovere = slice_correnti - target_slices
        slice_inizio = slice_da_rimuovere // 2
        slice_fine = slice_da_rimuovere - slice_inizio
        volume_ridotto = volume_3d[:, :, slice_inizio:slice_correnti-slice_fine]
        return volume_ridotto

    else:
        return volume_3d

# Funzione per suddividere sia la placca che la maschera
def suddividi_placca_maschera(volume_3d, maschera, dimensione_blocco=(32, 32, 32)):
    # Applica la maschera per estrarre solo la parte della placca
    placca = np.where(maschera > 0, volume_3d, 0)
    
    # Verifica se le dimensioni della placca e della maschera sono divisibili esattamente per il blocco
    shape_placca = np.array(placca.shape)
    shape_maschera = np.array(maschera.shape)
    if np.any(shape_placca % np.array(dimensione_blocco) != 0) or np.any(shape_maschera % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni della placca o della maschera non sono divisibili esattamente per il blocco 3D.")
    
    # Suddividi in blocchi sia la placca che la maschera
    blocchi_placca = view_as_blocks(placca, block_shape=dimensione_blocco)
    blocchi_maschera = view_as_blocks(maschera, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in liste di blocchi singoli
    blocchi_placca_risagomati = blocchi_placca.reshape(-1, *dimensione_blocco)
    blocchi_maschera_risagomati = blocchi_maschera.reshape(-1, *dimensione_blocco)
    
    return blocchi_placca_risagomati, blocchi_maschera_risagomati

## Features Area BLOCCHI

In [9]:

# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Cartelle da saltare
cartelle_da_saltare = []

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
blocchi_numeri = []
dimension_mismatch = []

# Imposta l'estrattore radiomico
estrattore = featureextractor.RadiomicsFeatureExtractor()


# Funzione per estrarre le features da ciascun blocco
def estrai_features_blocchi(blocchi, maschere):
    feature_vectors = []
    blocchi_successo = []
    feature_names = None
    
    for i, (blocco, maschera) in enumerate(zip(blocchi, maschere)):
        if not verifica_maschera_valida(maschera):
            continue
        
        if blocco.ndim < 3 or maschera.ndim < 3:
            continue
        
        sitk_blocco = numpy_to_sitk(blocco)
        sitk_maschera = numpy_to_sitk(maschera)
        
        try:
            results = estrattore.execute(sitk_blocco, sitk_maschera)
            if feature_names is None:
                feature_names = list(results.keys())
                
            feature_vector = [results.get(key, np.nan) for key in feature_names]
            feature_vectors.append(feature_vector)
            blocchi_successo.append(i)

            # Visualizza il blocco processato con successo
            #print(f"Visualizzazione del blocco {i+1} (processato con successo)")
            #visualizza_blocco(blocco)  # Visualizza la sezione centrale del blocco

        except ValueError as e:
            print(f"Errore durante l'estrazione delle features per il blocco {i}: {e}")
            continue

    return feature_vectors, blocchi_successo, feature_names


# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    if paziente in cartelle_da_saltare:
        print(f"Saltando paziente: {paziente}")
        continue

    print(f"Processando paziente: {paziente}")
    
    immagine_file = trova_nrrd_file(os.path.join(pazienti_dir, paziente))
    maschera_file = trova_nrrd_file(os.path.join(roi_dir, paziente))
    
    if immagine_file and maschera_file:
        volume_placca = carica_volume_paziente(os.path.join(pazienti_dir, paziente))
        volume_maschera = carica_volume_paziente(os.path.join(roi_dir, paziente))
        
        if volume_placca.shape == volume_maschera.shape:
            print(volume_placca.shape)
            print(volume_maschera.shape)
            if len(volume_placca.shape) == 4:
                volume_placca = np.squeeze(volume_placca, axis = -1)
                volume_maschera = np.squeeze(volume_maschera, axis = -1)

            if volume_placca.shape[2] != 512:
                volume_placca = riduci_slice(volume_placca)
                volume_maschera = riduci_slice(volume_maschera)

            # Suddividi sia la placca che la maschera
            blocchi_placca, blocchi_maschera = suddividi_placca_maschera(volume_placca, volume_maschera, dimensione_blocco=(16, 16, 16))
            
            # Estrai le features dai blocchi di placca e dalle corrispondenti maschere
            features_blocchi, blocchi_successo, feature_names = estrai_features_blocchi(blocchi_placca, blocchi_maschera)
            
            for blocco, features in zip(blocchi_successo, features_blocchi):
                feature_vectors.append(features)
                patient_names.append(paziente)
                blocchi_numeri.append(blocco)
        else:
            print(f"Dimensioni non corrispondenti per il paziente {paziente}")
            dimension_mismatch.append(paziente)
    else:
        print(f"File mancanti per il paziente {paziente}")

# Crea un DataFrame e salva come CSV
df = pd.DataFrame(feature_vectors, columns=feature_names)
df.insert(0, 'paziente', patient_names)
df.insert(1, 'blocco', blocchi_numeri)

# Salva il DataFrame in un file CSV
#df.to_csv('features_radiomiche_blocchi.csv', index=False)

Processando paziente: 100
(512, 512, 529, 1)
(512, 512, 529, 1)


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Errore durante l'estrazione delle features per il blocco 13708: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.
Processando paziente: 101
(512, 512, 537, 1)
(512, 512, 537, 1)


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Errore durante l'estrazione delle features per il blocco 21996: mask has too few dimensions (number of dimensions 1, minimum required 2)


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Errore durante l'estrazione delle features per il blocco 23055: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Processando paziente: 102
(512, 512, 586, 1)
(512, 512, 586, 1)


KeyboardInterrupt: 

## Salvataggio

In [6]:

# Carica il file CSV
file_path = 'features_radiomiche_blocchi.csv'
df = pd.read_csv(file_path)

# Stampa tutti i nomi delle colonne
print("Nomi delle colonne:")
print(df.columns.tolist())

# Specifica i nomi delle colonne da eliminare
colonne_da_eliminare = ['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass']  # Sostituisci con i nomi reali delle colonne da eliminare

# Elimina le colonne specificate
df = df.drop(columns=colonne_da_eliminare)

# Ordina il DataFrame in base alla prima colonna per numero in ordine crescente
df_sorted = df.sort_values(by=df.columns[0])

# Salva il DataFrame ordinato in un nuovo file CSV
df_sorted.to_csv('features_blocchi_nuove_16.csv', index=False)


Nomi delle colonne:
['paziente', 'blocco', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass', 'original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength', 'origin

In [25]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'features_radiomiche_blocchi.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

NameError: name 'results' is not defined

## Da Provare

Blocchi Sovrapposti (Overlapping Blocks):Utilizza una finestra mobile che si sovrappone parzialmente tra i blocchi. Questo è particolarmente utile se la placca è piccola e non riempie completamente i blocchi di dimensioni standard. L'overlapping aiuta a garantire che nessuna parte della placca venga trascurata.
Adaptive Windowing:

Finestra Adattativa: Anziché usare blocchi di dimensioni fisse, puoi utilizzare finestre adattative che si adattano alle dimensioni della placca. Ad esempio, puoi estrarre blocchi di dimensioni variabili che siano abbastanza grandi da contenere la placca, ma non troppo grandi da includere troppo background.
Segmentation and Bounding Box Extraction:

Bounding Box: Segmenta il volume in base alla ROI e applica una bounding box attorno alla placca. Successivamente, estrai blocchi 3D di dimensioni adeguate attorno a questa bounding box. Se la placca è molto piccola, puoi espandere leggermente la bounding box per garantire che il blocco contenga la placca e una certa area circostante.
Grid Search with Padding:

Ricerca a Griglia con Padding: Suddividi il volume in blocchi utilizzando una griglia regolare ma includi un padding extra intorno alla placca. Questo significa che i blocchi saranno leggermente più grandi per garantire che la placca sia ben inclusa in uno dei blocchi.
Dynamic Slicing:

Slicing Dinamico: Se la placca è molto piccola, potresti considerare di estrarre slice multiple attorno alla placca e combinarle per formare un blocco 3D. Ad esempio, estrai le slice immediatamente sopra e sotto la placca e poi aggregale per creare un blocco che contenga la placca e un'area circostante.
Voxel-Based Extraction:

Estrazione Basata sui Voxel: Analizza i voxel che appartengono alla placca e poi crea blocchi basati su questi voxel. Puoi estrarre solo i voxel di interesse e poi costruire blocchi 3D attorno a questi voxel specifici.
Resampling e Scaling:

Risoluzione Adatta: Se i blocchi sono troppo grandi rispetto alla placca, considera di fare un resampling del volume per aumentare la risoluzione e ridurre le dimensioni dei blocchi. Questo può aiutare a garantire che la placca venga inclusa in modo più dettagliato all'interno dei blocchi.